<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Forum-Issue-8523" data-toc-modified-id="Forum-Issue-8523-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Forum Issue 8523</a></span><ul class="toc-item"><li><span><a href="#Prerequisites" data-toc-modified-id="Prerequisites-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Prerequisites</a></span></li><li><span><a href="#Initialization" data-toc-modified-id="Initialization-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Initialization</a></span><ul class="toc-item"><li><span><a href="#Ensure-database-is-running" data-toc-modified-id="Ensure-database-is-running-1.2.1"><span class="toc-item-num">1.2.1&nbsp;&nbsp;</span>Ensure database is running</a></span></li><li><span><a href="#Download-and-install-additional-components." data-toc-modified-id="Download-and-install-additional-components.-1.2.2"><span class="toc-item-num">1.2.2&nbsp;&nbsp;</span>Download and install additional components.</a></span></li></ul></li><li><span><a href="#Connect-to-database-and-populate-test-data" data-toc-modified-id="Connect-to-database-and-populate-test-data-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Connect to database and populate test data</a></span></li><li><span><a href="#Open-a-Terminal-Tab" data-toc-modified-id="Open-a-Terminal-Tab-1.4"><span class="toc-item-num">1.4&nbsp;&nbsp;</span>Open a Terminal Tab</a></span></li></ul></li><li><span><a href="#The-Problem:-No-Error-for-Non-Existent-UDF" data-toc-modified-id="The-Problem:-No-Error-for-Non-Existent-UDF-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>The Problem: No Error for Non-Existent UDF</a></span></li><li><span><a href="#Create-UDF" data-toc-modified-id="Create-UDF-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Create UDF</a></span></li><li><span><a href="#Register-UDF" data-toc-modified-id="Register-UDF-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Register UDF</a></span></li><li><span><a href="#Execute-UDF-(Works-as-Expected)" data-toc-modified-id="Execute-UDF-(Works-as-Expected)-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Execute UDF (Works as Expected)</a></span></li><li><span><a href="#Register-Again---Without-Change" data-toc-modified-id="Register-Again---Without-Change-6"><span class="toc-item-num">6&nbsp;&nbsp;</span>Register Again - Without Change</a></span></li><li><span><a href="#Register-Again---With-Change" data-toc-modified-id="Register-Again---With-Change-7"><span class="toc-item-num">7&nbsp;&nbsp;</span>Register Again - With Change</a></span></li><li><span><a href="#Clean-up" data-toc-modified-id="Clean-up-8"><span class="toc-item-num">8&nbsp;&nbsp;</span>Clean up</a></span></li><li><span><a href="#Further-Exploration-and-Resources" data-toc-modified-id="Further-Exploration-and-Resources-9"><span class="toc-item-num">9&nbsp;&nbsp;</span>Further Exploration and Resources</a></span></li><li><span><a href="#Next-steps" data-toc-modified-id="Next-steps-10"><span class="toc-item-num">10&nbsp;&nbsp;</span>Next steps</a></span></li></ul></div>

# Forum Issue 8523
Here is the link to the [Forum Issue](https://discuss.aerospike.com/t/calling-non-existent-udf-module-doesnt-throw-error/8523).
This notebook requires Aerospike datbase running on localhost. Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) for additional details and the docker container.

## Prerequisites
This tutorial assumes familiarity with the following topics:
- [Aerospike Notebooks - Readme and Tips](../readme_tips.ipynb)
- [Hello World](hello_world.ipynb)

## Initialization

### Ensure database is running
This notebook requires that Aerospike datbase is running. 

In [1]:
import io.github.spencerpark.ijava.IJava;
import io.github.spencerpark.jupyter.kernel.magic.common.Shell;
IJava.getKernelInstance().getMagics().registerMagics(Shell.class);
%sh asd

### Download and install additional components.
Install the Java client.

In [2]:
%%loadFromPOM
<dependencies>
  <dependency>
    <groupId>com.aerospike</groupId>
    <artifactId>aerospike-client</artifactId>
    <version>5.0.0</version>
  </dependency>
</dependencies>

## Connect to database and populate test data
The test data has 10 records with user-key "id-1" through "id-10", two bins (fields) "bin1" (1-10) and "bin2" ("1001"-"1010") in the namespace "test" and set "forum-8523". 

In [3]:
import com.aerospike.client.AerospikeClient;
import com.aerospike.client.Bin;
import com.aerospike.client.Key;
import com.aerospike.client.policy.WritePolicy;

AerospikeClient client = new AerospikeClient("localhost", 3000);
System.out.println("Initialized the client and connected to the cluster.");

String Namespace = "test";
String Set = "forum-8523";

WritePolicy wpolicy = new WritePolicy();
wpolicy.sendKey = true;
for (int i = 1; i <= 10; i++) {
    Key key = new Key(Namespace, Set, "id-"+i);
    Bin bin1 = new Bin(new String("bin1"), i);
    Bin bin2 = new Bin(new String("bin2"), 1000+i);
    Bin bin3 = new Bin(new String("bin3"), Integer.toString(1000+i));
    client.put(wpolicy, key, bin1, bin2, bin3);
}
System.out.format("Inserted 10 records in ns=%s set=%s.\n", Namespace, Set);


Initialized the client and connected to the cluster.
Inserted 10 records in ns=test set=forum-8523.


java.io.PrintStream@243f7748

## Open a Terminal Tab
In this tutorial, you will be executing many shell commands including Aerospike tools like adadm and aql in a terminal. Open a terminal tab by selecting File->Open from the notebook menu, and then New->Terminal.

# The Problem: No Error for Non-Existent UDF 
No exception is generated for a non-existent UDF.

In [4]:
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.task.ExecuteTask;
import com.aerospike.client.Value;
import com.aerospike.client.lua.LuaCache;

Statement stmt = new Statement();
stmt.setNamespace(Namespace);
stmt.setSetName(Set);
WritePolicy policy = new WritePolicy();
policy.filterExp = Exp.build(
    Exp.and(
        Exp.ge(Exp.intBin("bin1"), Exp.val(4)),
        Exp.le(Exp.intBin("bin1"), Exp.val(7))));
policy.maxRetries = 0;

ExecuteTask task = client.execute(policy, stmt, "NonExistent", "doesnt_exist",
                                 Value.get("bin2"), Value.get(1), Value.get("bin3"), Value.get("xo")); 
task.waitTillComplete(3000, 3000);

Examine the logs in the terminal tab. No error is logged.
<pre>
tail -20 /opt/aerospike/aerospike.log
</pre>

# Create UDF
Examine the following Lua code that updates two bins by adding and appending the values provided. 
<pre>
-- update the specified bins by adding and appending the values provided
function add_append(rec, binName1, addVal, binName2, appendVal)
    rec[binName1] = rec[binName1] + addVal
    rec[binName2] = rec[binName2] .. appendVal
    aerospike:update(rec)
end
</pre>

# Register UDF
Note, the UDF function is expected to be in "update_example.lua" file under "udf" directory.

Register the UDF with the server by executing the following code cell. The function invalidates the cache, removes the currently registered module, and registers the latest version.

In [5]:
import com.aerospike.client.policy.Policy;
import com.aerospike.client.task.RegisterTask;
import com.aerospike.client.Language;
import com.aerospike.client.lua.LuaConfig;
import com.aerospike.client.lua.LuaCache;

LuaConfig.SourceDirectory = "../udf";
String UDFModule = "update_example";
String UDFFile = "update_example.lua";

void registerUDF() {
    // clear the lua cache
    LuaCache.clearPackages();
    Policy policy = new Policy();
    // remove the current module, if any
    client.removeUdf(null, UDFFile);
    RegisterTask task = client.register(policy, LuaConfig.SourceDirectory+"/"+UDFFile, 
                                        UDFFile, Language.LUA);
    task.waitTillComplete();
    System.out.format("Registered the UDF module %s.", UDFFile);;
}

registerUDF();

Registered the UDF module update_example.lua.

# Execute UDF (Works as Expected)
Run the UDF on multiple records using an expression filter to specify records with bin1 values in the range 4-7 (both inclusive).

In [6]:
import com.aerospike.client.query.Statement;
import com.aerospike.client.query.Filter;
import com.aerospike.client.exp.Exp;
import com.aerospike.client.task.ExecuteTask;
import com.aerospike.client.Value;

Statement stmt = new Statement();
stmt.setNamespace(Namespace);
stmt.setSetName(Set);
WritePolicy policy = new WritePolicy();
policy.filterExp = Exp.build(
    Exp.and(
        Exp.ge(Exp.intBin("bin1"), Exp.val(4)),
        Exp.le(Exp.intBin("bin1"), Exp.val(7))));
policy.maxRetries = 0;

ExecuteTask task = client.execute(policy, stmt, UDFModule, "add_append",
                                 Value.get("bin2"), Value.get(1), Value.get("bin3"), Value.get("xo")); 
task.waitTillComplete(3000, 3000);

Examine the logs in the terminal tab. No error (as expected)
<pre>
tail -20 /opt/aerospike/aerospike.log
</pre>

Examine the database content in the terminal window. The bin values should be changed.
<pre>
aql -c "select * from test.forum-8523"
</pre>

# Register Again - Without Change
Re-register without removing the existing UDF first. Note there is no exception or error.

Executing the UDF works as expected.

In [7]:
RegisterTask task = client.register(policy, LuaConfig.SourceDirectory+"/"+UDFFile, 
                                    UDFFile, Language.LUA);
task.waitTillComplete();
System.out.format("Registered the UDF module %s.", UDFFile);;

Statement stmt = new Statement();
stmt.setNamespace(Namespace);
stmt.setSetName(Set);
WritePolicy policy = new WritePolicy();
policy.filterExp = Exp.build(
    Exp.and(
        Exp.ge(Exp.intBin("bin1"), Exp.val(4)),
        Exp.le(Exp.intBin("bin1"), Exp.val(7))));
policy.maxRetries = 0;

ExecuteTask task = client.execute(policy, stmt, UDFModule, "add_append",
                                 Value.get("bin2"), Value.get(1), Value.get("bin3"), Value.get("xo")); 
task.waitTillComplete();

Registered the UDF module update_example.lua.

Examine the database content in the terminal window. The bin values should be changed.

aql -c "select * from test.forum-8523"

# Register Again - With Change
Change the UDF as follows, save, and then register without removing the existing UDF. 
<pre>
-- update the specified bins by adding and appending the values provided
function add_append(rec, binName1, addVal, binName2, appendVal)
    rec[binName1] = rec[binName1] + addVal
    rec[binName2] = rec[binName2] .. appendVal .. addVal
    aerospike:update(rec)
end
</pre>
Execute the UDF and note the new version is in effect.

In [10]:
RegisterTask task = client.register(policy, LuaConfig.SourceDirectory+"/"+UDFFile, 
                                    UDFFile, Language.LUA);
task.waitTillComplete();
System.out.format("Registered the UDF module %s.", UDFFile);;

Statement stmt = new Statement();
stmt.setNamespace(Namespace);
stmt.setSetName(Set);
WritePolicy policy = new WritePolicy();
policy.filterExp = Exp.build(
    Exp.and(
        Exp.ge(Exp.intBin("bin1"), Exp.val(4)),
        Exp.le(Exp.intBin("bin1"), Exp.val(7))));
policy.maxRetries = 0;

ExecuteTask task = client.execute(policy, stmt, UDFModule, "add_append",
                                 Value.get("bin2"), Value.get(1), Value.get("bin3"), Value.get("xo")); 
task.waitTillComplete();

Registered the UDF module update_example.lua.

Examine the database content in the terminal window. The bin values should be changed.

aql -c "select * from test.forum-8523"

# Clean up
Remove tutorial data and close connection.

In [ ]:
client.truncate(null, Namespace, null, null);
client.close();
System.out.println("Removed tutorial data and closed server connection.");

# Further Exploration and Resources
Here are some links for further exploration

Resources
- Aerospike Developer Hub
    - [Java Developers Resources](https://developer.aerospike.com/java-developers)
- Github repos
    - [Java code examples](https://github.com/aerospike/aerospike-client-java/tree/master/examples/src/com/aerospike/examples)
- Documentation
    - [Java Client](https://www.aerospike.com/docs/client/java/index.html)
    - [Java API Reference](https://www.aerospike.com/apidocs/java/)

# Next steps

Visit [Aerospike notebooks repo](https://github.com/aerospike-examples/interactive-notebooks) to run additional Aerospike notebooks. To run a different notebook, download the notebook from the repo to your local machine, and then click on File->Open in the notebook menu, and select Upload.